# Convert OFF segmentation files to VTK format

In [1]:
import os
import os.path
import pandas as pd
import sys
import vtk
import off2vtk
import numpy as np

## Setup input and output

In [2]:
# Check the dirs
#INPUT_PATH = '/Users/williamengelhardt/Documents/Yale_Lab_Summer/conversion/mask_left'
INPUT_PATH = "/Volumes/William's Passport SSD/Yale_Data/data"

print('INPUT_PATH: %s, exists=%s' % 
      (INPUT_PATH, os.path.exists(INPUT_PATH)))

INPUT_PATH: /Volumes/William's Passport SSD/Yale_Data/data, exists=True


In [3]:
# Check the output dirs
#OUTPUT_PATH = r'/Users/Tom/Desktop/Deep-Learning/conversion/off'
#OUTPUT_PATH = '/Users/williamengelhardt/Documents/Yale_Lab_Summer/conversion/PVE_data/segmentation_left_liver'
OUTPUT_PATH = "/Volumes/William's Passport SSD/Yale_Data/data"

print('OUTPUT_PATH: %s, exists=%s' % 
      (OUTPUT_PATH, os.path.exists(OUTPUT_PATH)))

OUTPUT_PATH: /Volumes/William's Passport SSD/Yale_Data/data, exists=True


## Identify all .off files

Asssumes that all data is in the format:

`INPUT_PATH/ACCESSION_ID/SUBFOLDER/SEGM_NAME.off`

In [5]:
def create_accession_id(row):
    root = os.path.split(row['INPUT_PATH'])[0]
    value = os.path.split(root)[0]
    value2 = os.path.split(value)[-1]
    return value2

def create_segm_name(row):
    name = os.path.split(row['INPUT_PATH'])[1]
    value = os.path.splitext(name)[0]
    return value


df = pd.DataFrame(columns=['INPUT_PATH'])

# 1. Get all the input
input_paths = []
for root, dirs, files in os.walk(INPUT_PATH):
    # Get all files in the input dir
    for f in files:        
        # Parse the .off files
        if os.path.splitext(f)[1] == '.off':
            input_paths.append(os.path.join(root,f))

df['INPUT_PATH'] = input_paths

# 2. Parse the ACCESSION_ID and SEGM_NAME
df['ACCESSION_ID'] = df.apply(lambda row: create_accession_id(row), axis=1)
df['SEGM_NAME'] = df.apply(lambda row: create_segm_name(row), axis=1)


df.describe()

,INPUT_PATH,ACCESSION_ID,SEGM_NAME
count,571,571,571
unique,571,142,323
top,/Volumes/William's Passport SSD/Yale_Data/data...,m_41,p45_total_post
freq,1,8,2


## Export the off files

In [6]:
for idx, row in df.iterrows():
    SUBFOLDER = os.path.split(os.path.split(row['INPUT_PATH'])[0])[-1]
    output_path1 = os.path.join(OUTPUT_PATH, row['ACCESSION_ID'])
    if not os.path.exists(output_path1):        
        try:
            os.mkdir(output_path1)
        except :
            print('ERROR: Could not create dir')            
        else:
            print('Created new output dir: %s' % output_path1)
            
    output_path2 = os.path.join(output_path1, SUBFOLDER)
    if not os.path.exists(output_path2):        
        try:
            os.mkdir(output_path2)
        except :
            print('ERROR: Could not create dir')            
        else:
            print('Created new output dir: %s' % output_path2)
    
    # Copy the segm data over
    segm_src_path = os.path.join(row['INPUT_PATH'])
    if os.path.exists(segm_src_path) and os.path.exists(output_path2):
        segm_dst_path = os.path.join(output_path2, '%s.vtk' % row['SEGM_NAME'])
        
        if not os.path.exists(segm_dst_path):
            vtk_poly = off2vtk.convert(segm_src_path)

            print('Saving VTK file: %s' % segm_dst_path)
            writer = vtk.vtkPolyDataWriter()
            writer.SetFileName(segm_dst_path)
            writer.SetInputData(vtk_poly)
            writer.Update()

Surface has 5321 points and 10638 faces
Saving VTK file: /Volumes/William's Passport SSD/Yale_Data/data/a_0/post/liver_left_mdacc_p_0.vtk
Surface has 5648 points and 11296 faces
Saving VTK file: /Volumes/William's Passport SSD/Yale_Data/data/a_0/post/liver_total_mdacc_p_0.vtk
Surface has 5448 points and 10892 faces
Saving VTK file: /Volumes/William's Passport SSD/Yale_Data/data/a_0/pre/liver_left_mdacc_p_0.vtk
Surface has 4760 points and 9520 faces
Saving VTK file: /Volumes/William's Passport SSD/Yale_Data/data/a_0/pre/liver_total_mdacc_p_0.vtk
Surface has 4931 points and 9858 faces
Saving VTK file: /Volumes/William's Passport SSD/Yale_Data/data/a_1/post/liver_left_mdacc_p_1.vtk
Surface has 5838 points and 11672 faces
Saving VTK file: /Volumes/William's Passport SSD/Yale_Data/data/a_1/post/liver_total_mdacc_p_1.vtk
Surface has 1332 points and 2660 faces
Saving VTK file: /Volumes/William's Passport SSD/Yale_Data/data/a_1/pre/liver_left_mdacc_p_1.vtk
Surface has 5302 points and 10602 fac